In [1]:
import os


In [2]:
%pwd


'c:\\Users\\Lenovo\\Desktop\\stb_pfe_mlflow\\research'

In [3]:
os.chdir("../")

In [27]:
%pwd

'c:\\Users\\Lenovo\\Desktop\\stb_pfe_mlflow'

In [28]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/MedAliRommene/stb_pfe_mlflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="MedAliRommene"
os.environ["MLFLOW_TRACKING_PASSWORD"]="5e389dbd0f1b00486b1dcbc772a112212c63eff6"

In [29]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [30]:
from stb_pfe_mlflow.constants import *
from stb_pfe_mlflow.utils.common import read_yaml, create_directories

In [31]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.GradientBoostingClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/MedAliRommene/stb_pfe_mlflow.mlflow",
           
        )

        return model_evaluation_config


In [32]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [45]:
import pandas as pd
import joblib
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import mlflow
from urllib.parse import urlparse
from pathlib import Path
from stb_pfe_mlflow.utils.common import save_json

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        # Load test data
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        # Extract target and features
        test_y = test_data[self.config.target_column]
        test_x = test_data.drop(columns=["tiers_key", "Cluster"])  # drop 'tiers_key' and target column 'Cluster'

        # new encoders and scalers saved during training
        label_encoder = LabelEncoder()
        scaler = StandardScaler()
        

        # Encode categorical features in the same way as in training
        for col in test_x.select_dtypes(include=['object']).columns:
            # Fit the LabelEncoder on the unique values of the current column
            label_encoder.fit(test_x[col].astype(str))  # Fitting the encoder on the current test column
            test_x[col] = label_encoder.transform(test_x[col].astype(str))
        # Encode the target labels (ensure it matches the training labels)
        test_y_encoded = label_encoder.fit_transform(test_y)

        # Standardize the features as in the training step
        test_x_scaled = scaler.fit_transform(test_x)  # Fit the scaler on the test features


        # Set MLflow tracking
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            # Predict using the loaded model
            predicted_qualities = model.predict(test_x_scaled)

            # Evaluate metrics
            (rmse, mae, r2) = self.eval_metrics(test_y_encoded, predicted_qualities)

            # Saving metrics locally
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            # Log parameters and metrics in MLflow
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                mlflow.sklearn.log_model(
                    model, "model", registered_model_name="GradientBoostingClassifier"
                )
            else:
                mlflow.sklearn.log_model(model, "model")

# Note: Ensure you have paths for 'label_encoder_path' and 'scaler_path' in your ModelEvaluationConfig.


In [46]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-10-05 23:17:56,250: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-05 23:17:56,253: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-05 23:17:56,257: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-10-05 23:17:56,259: INFO: common: created directory at: artifacts]
[2024-10-05 23:17:56,260: INFO: common: created directory at: artifacts/model_evaluation]
[2024-10-05 23:17:56,704: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


c:\Users\Lenovo\Desktop\stb_pfe_mlflow\.venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'GradientBoostingClassifier'.
2024/10/05 23:18:08 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: GradientBoostingClassifier, version 1
Created version '1' of model 'GradientBoostingClassifier'.
